In [2]:
import numpy as np
import pandas as pd
import keras

In [ ]:
def _bn_relu(input):
    from keras.layers.normalization import BatchNormalization
    from keras.layers import Activation
    norm = BatchNormalization(axis = 2)(input)
    return Activation("relu")(norm)


def _bn_relu_conv(**conv_params):
    from keras.regularizers import l2
    from keras.layers import Conv2D
    filters = conv_params['filters']
    kernel_size = conv_params['kernel_size']
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1e-4))
    
    def f(input):
        activation = _bn_relu(input)
        return Conv2D(filters = filters, kernel_size = kernel_size,
                     strides = strides, padding = padding,
                     kernel_initializer = kernel_initializer,
                     kernel_regularizer = kernel_regularizer)(activation)
    
    return f


def _shortcut(inptu, residual):
    import keras.backend as K
    from keras.layers import conv2D
    from keras.layers.merge import add
    input_shape = K.int_shape(input)
    residual_shape = K.int_shape(residual)
    stride_width = int(round(input_shape[0] / residual_shape[0]))
    stride_height = int(round(input_shape[1] / residual_shape[1]))
    equal_channels = input_shape[2] == residual_shape[2]
    
    shortcut = input
    # 1 * 1 conv if shape is different. Else identity
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        shortcut = Conv2D(filters = residual_shape[2],
                         kernel_size = (1, 1),
                         strides = (stride_width, stride_height),
                         padding = "valid",
                         kernel_initializer = "he_normal",
                         kernel_regularizer = l2(0.0001))(input)
    return add([shortcut, residual])
    
    
def basic_block(filters, init_strides = (1, 1), is_first_block_of_first_layer = False):
    from keras.regularizers import l2
    form keras.layers import Conv2D
    def f(input):
        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just dit bn->relu->maxpool
            conv1 = Conv2D(fitlers = filters, kernel_size = (3, 3),
                          strides = init_strides,
                          padding = "same",
                          kernel_initializer = "he_normal",
                          kernel_regularizer = l2(1e-4))(input)
        else:
            conv1 = _bn_relu_conv(fiters = filters, kernel_size = (3, 3),
                                 strides = init_strides)(input)

        residual = _bn_relu_conv(filters = filters, kernel_size = (3, 3))(conv1)
        return _shortcut(input, residual)
    
    return f


def _residual_block(filters, repetitions, is_first_layer = False):
    def f(input):
        for i in range(repetitions):
            init_strides = (1, 1)
            if i == 0 and not is_first_layer:
                init_strides = (2, 2)
            input = basic_block(filters = filters, init_strides = init_strides, 
                                       is_first_block_of_first_layer = (is_first_layer and i == 0))(input)
        return input
    
    return f
        
        
def build(input_shape, num_outputs):
    from keras.layers import Input
    import keras.backend as K
    input = Input(shape = input_shape)
    conv1 = _conv_bn_relu(filters = 64, kerenls = (5, 5), strides = (2, 2))(input)
    pool1 = MaxPooling2D(pool_size = (3, 3), strides = (2, 2), padding = "same")(conv1)
    
    block = pool1
    filters = 64
    for i, r in enumerate([2, 2, 2, 2]):
        block = _residual_block(filters = filters, repetitions = r, is_first_layer = (i == 0))(block)
        filters *= 2
        
    block = _bn_relu(block)
    
    # Classifier block
    block_shape = K.int_shape(block)
    pool2 = AveragePooling2D(pool_size = (block_shape[0], block_shape[1]), strides = (1, 1))(block)
    flatten1 = Flatten()(pool2)
    dense = Dense(num_outputs, kernel_initializer = "he_normal", activation = "softmax")(flatten1)
    model = Model(inputs = input, outputs = dense)
    return model